# GAN IMPLEMENTATION ON MNIST DATASET

In this tutorial we’ll be building a generative adversarial network (GAN) trained on the MNIST dataset.The purpose of this tutorial is to learn how to create undistinguishable images of hand-written digits using GAN. Let's start from the beginning by importing all the required libraries and by defining some hyper-parameters which is later used.

In [2]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pylab
import numpy as np

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 30
batch_size = 32

In [3]:
# Image processing
# Image processing
transform = transforms.Compose([
transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

The above transformations are necessary to make the image compatible as an input to the neural network of the discriminator. Here, we will be using MNIST dataset consisting of 28x28 black and white images. Download the dataset here.

In [8]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='./data/',
                                   train=True,
                                   transform = transforms.Compose([
                    transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]),
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [9]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())


In [ ]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

# Statistics to be saved
d_losses = np.zeros(num_epochs)
g_losses = np.zeros(num_epochs)
real_scores = np.zeros(num_epochs)
fake_scores = np.zeros(num_epochs)

# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.view(batch_size, -1)
        images = Variable(images)
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1)
        real_labels = Variable(real_labels)
        fake_labels = torch.zeros(batch_size, 1)
        fake_labels = Variable(fake_labels)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size)
        z = Variable(z)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        # If D is trained so well, then don't update
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size)
        z = Variable(z)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        # if G is trained so well, then don't update
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        # =================================================================== #
        #                          Update Statistics                          #
        # =================================================================== #
        d_losses[epoch] = d_losses[epoch]*(i/(i+1.)) + d_loss.data*(1./(i+1.))
        g_losses[epoch] = g_losses[epoch]*(i/(i+1.)) + g_loss.data*(1./(i+1.))
        real_scores[epoch] = real_scores[epoch]*(i/(i+1.)) + real_score.mean().data*(1./(i+1.))
        fake_scores[epoch] = fake_scores[epoch]*(i/(i+1.)) + fake_score.mean().data*(1./(i+1.))
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.data, g_loss.data, 
                          real_score.mean().data, fake_score.mean().data))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.view(images.size(0), 1, 28, 28)
        save_image(denorm(images.data), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images.data), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
    
    # Save and plot Statistics
    np.save(os.path.join(save_dir, 'd_losses.npy'), d_losses)
    np.save(os.path.join(save_dir, 'g_losses.npy'), g_losses)
    np.save(os.path.join(save_dir, 'fake_scores.npy'), fake_scores)
    np.save(os.path.join(save_dir, 'real_scores.npy'), real_scores)
    
    plt.figure()
    pylab.xlim(0, num_epochs + 1)
    plt.plot(range(1, num_epochs + 1), d_losses, label='d loss')
    plt.plot(range(1, num_epochs + 1), g_losses, label='g loss')    
    plt.legend()
    plt.savefig(os.path.join(save_dir, 'loss.pdf'))
    plt.close()

    plt.figure()
    pylab.xlim(0, num_epochs + 1)
    pylab.ylim(0, 1)
    plt.plot(range(1, num_epochs + 1), fake_scores, label='fake score')
    plt.plot(range(1, num_epochs + 1), real_scores, label='real score')    
    plt.legend()
    plt.savefig(os.path.join(save_dir, 'accuracy.pdf'))
    plt.close()

    # Save model at checkpoints
    if (epoch+1) % 50 == 0:
        torch.save(G.state_dict(), os.path.join(save_dir, 'G--{}.ckpt'.format(epoch+1)))
        torch.save(D.state_dict(), os.path.join(save_dir, 'D--{}.ckpt'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pylab
import numpy as np

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 300
batch_size = 32
sample_dir = 'samples'
save_dir = 'save'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

if not os.path.exists(save_dir):
    os.makedirs(save_dir)



# MNIST dataset
mnist = torchvision.datasets.MNIST(root='./data/',
                                   train=True,
                                   transform = transforms.Compose([
transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]))

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())


# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

# Statistics to be saved
d_losses = np.zeros(num_epochs)
g_losses = np.zeros(num_epochs)
real_scores = np.zeros(num_epochs)
fake_scores = np.zeros(num_epochs)

# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.view(batch_size, -1)
        images = Variable(images)
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1)
        real_labels = Variable(real_labels)
        fake_labels = torch.zeros(batch_size, 1)
        fake_labels = Variable(fake_labels)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size)
        z = Variable(z)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        # If D is trained so well, then don't update
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size)
        z = Variable(z)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        # if G is trained so well, then don't update
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        # =================================================================== #
        #                          Update Statistics                          #
        # =================================================================== #
        d_losses[epoch] = d_losses[epoch]*(i/(i+1.)) + d_loss.data*(1./(i+1.))
        g_losses[epoch] = g_losses[epoch]*(i/(i+1.)) + g_loss.data*(1./(i+1.))
        real_scores[epoch] = real_scores[epoch]*(i/(i+1.)) + real_score.mean().data*(1./(i+1.))
        fake_scores[epoch] = fake_scores[epoch]*(i/(i+1.)) + fake_score.mean().data*(1./(i+1.))
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.data, g_loss.data, 
                          real_score.mean().data, fake_score.mean().data))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.view(images.size(0), 1, 28, 28)
        save_image(denorm(images.data), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images.data), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
    
    # Save and plot Statistics
    np.save(os.path.join(save_dir, 'd_losses.npy'), d_losses)
    np.save(os.path.join(save_dir, 'g_losses.npy'), g_losses)
    np.save(os.path.join(save_dir, 'fake_scores.npy'), fake_scores)
    np.save(os.path.join(save_dir, 'real_scores.npy'), real_scores)
    
    plt.figure()
    pylab.xlim(0, num_epochs + 1)
    plt.plot(range(1, num_epochs + 1), d_losses, label='d loss')
    plt.plot(range(1, num_epochs + 1), g_losses, label='g loss')    
    plt.legend()
    plt.savefig(os.path.join(save_dir, 'loss.pdf'))
    plt.close()

    plt.figure()
    pylab.xlim(0, num_epochs + 1)
    pylab.ylim(0, 1)
    plt.plot(range(1, num_epochs + 1), fake_scores, label='fake score')
    plt.plot(range(1, num_epochs + 1), real_scores, label='real score')    
    plt.legend()
    plt.savefig(os.path.join(save_dir, 'accuracy.pdf'))
    plt.close()

    # Save model at checkpoints
    if (epoch+1) % 50 == 0:
        torch.save(G.state_dict(), os.path.join(save_dir, 'G--{}.ckpt'.format(epoch+1)))
        torch.save(D.state_dict(), os.path.join(save_dir, 'D--{}.ckpt'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [0/300], Step [200/1875], d_loss: 0.0370, g_loss: 4.5494, D(x): 0.99, D(G(z)): 0.03
Epoch [0/300], Step [400/1875], d_loss: 0.0738, g_loss: 4.6561, D(x): 1.00, D(G(z)): 0.07
Epoch [0/300], Step [600/1875], d_loss: 0.0586, g_loss: 4.5380, D(x): 0.97, D(G(z)): 0.03
Epoch [0/300], Step [800/1875], d_loss: 0.0605, g_loss: 5.0078, D(x): 0.99, D(G(z)): 0.04
Epoch [0/300], Step [1000/1875], d_loss: 0.1566, g_loss: 3.3071, D(x): 0.99, D(G(z)): 0.13
Epoch [0/300], Step [1200/1875], d_loss: 0.4650, g_loss: 3.0979, D(x): 0.86, D(G(z)): 0.17
Epoch [0/300], Step [1400/1875], d_loss: 0.1799, g_loss: 4.1657, D(x): 0.91, D(G(z)): 0.05
Epoch [0/300], Step [1600/1875], d_loss: 0.4428, g_loss: 2.7547, D(x): 0.90, D(G(z)): 0.26
Epoch [0/300], Step [1800/1875], d_loss: 0.1531, g_loss: 3.9611, D(x): 0.90, D(G(z)): 0.03
Epoch [1/300], Step [200/1875], d_loss: 0.1383, g_loss: 4.5227, D(x): 0.92, D(G(z)): 0.02
Epoch [1/300], Step [400/1875], d_loss: 0.6755, g_loss: 2.1854, D(x): 0.87, D(G(z)): 0.28
Epoch

Epoch [10/300], Step [400/1875], d_loss: 0.4935, g_loss: 3.1192, D(x): 0.77, D(G(z)): 0.08
Epoch [10/300], Step [600/1875], d_loss: 0.7113, g_loss: 4.6690, D(x): 0.95, D(G(z)): 0.41
Epoch [10/300], Step [800/1875], d_loss: 0.7114, g_loss: 2.2582, D(x): 0.79, D(G(z)): 0.15
Epoch [10/300], Step [1000/1875], d_loss: 0.5554, g_loss: 2.4717, D(x): 0.79, D(G(z)): 0.12
Epoch [10/300], Step [1200/1875], d_loss: 0.4279, g_loss: 3.5602, D(x): 0.89, D(G(z)): 0.15
Epoch [10/300], Step [1400/1875], d_loss: 0.5294, g_loss: 2.1114, D(x): 0.89, D(G(z)): 0.23
Epoch [10/300], Step [1600/1875], d_loss: 0.7233, g_loss: 2.4193, D(x): 0.70, D(G(z)): 0.05
Epoch [10/300], Step [1800/1875], d_loss: 0.6258, g_loss: 3.0646, D(x): 0.76, D(G(z)): 0.14
Epoch [11/300], Step [200/1875], d_loss: 0.5859, g_loss: 2.7957, D(x): 0.75, D(G(z)): 0.12
Epoch [11/300], Step [400/1875], d_loss: 0.7963, g_loss: 3.1875, D(x): 0.91, D(G(z)): 0.32
Epoch [11/300], Step [600/1875], d_loss: 0.7191, g_loss: 4.2619, D(x): 0.72, D(G(z)):

Epoch [20/300], Step [400/1875], d_loss: 0.5384, g_loss: 1.3235, D(x): 0.88, D(G(z)): 0.28
Epoch [20/300], Step [600/1875], d_loss: 0.5616, g_loss: 2.7775, D(x): 0.87, D(G(z)): 0.28
Epoch [20/300], Step [800/1875], d_loss: 0.5341, g_loss: 2.2629, D(x): 0.74, D(G(z)): 0.15
Epoch [20/300], Step [1000/1875], d_loss: 0.6396, g_loss: 3.8038, D(x): 0.74, D(G(z)): 0.12
Epoch [20/300], Step [1200/1875], d_loss: 0.3951, g_loss: 2.7118, D(x): 0.92, D(G(z)): 0.23
Epoch [20/300], Step [1400/1875], d_loss: 0.7331, g_loss: 1.9213, D(x): 0.75, D(G(z)): 0.20
Epoch [20/300], Step [1600/1875], d_loss: 0.4560, g_loss: 3.1314, D(x): 0.79, D(G(z)): 0.09
Epoch [20/300], Step [1800/1875], d_loss: 0.4140, g_loss: 3.8557, D(x): 0.82, D(G(z)): 0.09
Epoch [21/300], Step [200/1875], d_loss: 0.4898, g_loss: 2.7396, D(x): 0.85, D(G(z)): 0.17
Epoch [21/300], Step [400/1875], d_loss: 0.5534, g_loss: 2.1471, D(x): 0.94, D(G(z)): 0.28
Epoch [21/300], Step [600/1875], d_loss: 0.5996, g_loss: 2.7626, D(x): 0.89, D(G(z)):

Epoch [30/300], Step [400/1875], d_loss: 0.9485, g_loss: 2.4929, D(x): 0.67, D(G(z)): 0.26
Epoch [30/300], Step [600/1875], d_loss: 0.6935, g_loss: 1.6262, D(x): 0.70, D(G(z)): 0.19
Epoch [30/300], Step [800/1875], d_loss: 0.8618, g_loss: 1.8299, D(x): 0.76, D(G(z)): 0.38
Epoch [30/300], Step [1000/1875], d_loss: 0.9552, g_loss: 1.7295, D(x): 0.82, D(G(z)): 0.37
Epoch [30/300], Step [1200/1875], d_loss: 1.1623, g_loss: 1.1625, D(x): 0.72, D(G(z)): 0.44
Epoch [30/300], Step [1400/1875], d_loss: 0.8555, g_loss: 2.5272, D(x): 0.60, D(G(z)): 0.09
Epoch [30/300], Step [1600/1875], d_loss: 0.8105, g_loss: 1.1122, D(x): 0.73, D(G(z)): 0.27
Epoch [30/300], Step [1800/1875], d_loss: 0.8215, g_loss: 1.8619, D(x): 0.73, D(G(z)): 0.27
Epoch [31/300], Step [200/1875], d_loss: 0.9432, g_loss: 0.9645, D(x): 0.65, D(G(z)): 0.26
Epoch [31/300], Step [400/1875], d_loss: 0.8737, g_loss: 1.9912, D(x): 0.72, D(G(z)): 0.31
Epoch [31/300], Step [600/1875], d_loss: 1.1171, g_loss: 1.4398, D(x): 0.73, D(G(z)):

Epoch [40/300], Step [400/1875], d_loss: 0.7123, g_loss: 1.8061, D(x): 0.75, D(G(z)): 0.29
Epoch [40/300], Step [600/1875], d_loss: 0.9869, g_loss: 1.1221, D(x): 0.81, D(G(z)): 0.43
Epoch [40/300], Step [800/1875], d_loss: 1.0844, g_loss: 1.9893, D(x): 0.68, D(G(z)): 0.34
Epoch [40/300], Step [1000/1875], d_loss: 0.8743, g_loss: 2.1740, D(x): 0.66, D(G(z)): 0.19
Epoch [40/300], Step [1200/1875], d_loss: 0.7019, g_loss: 1.6076, D(x): 0.75, D(G(z)): 0.26
Epoch [40/300], Step [1400/1875], d_loss: 0.8169, g_loss: 1.3468, D(x): 0.74, D(G(z)): 0.33
Epoch [40/300], Step [1600/1875], d_loss: 1.0987, g_loss: 2.0973, D(x): 0.77, D(G(z)): 0.43
Epoch [40/300], Step [1800/1875], d_loss: 0.7448, g_loss: 1.7184, D(x): 0.73, D(G(z)): 0.26
Epoch [41/300], Step [200/1875], d_loss: 0.8337, g_loss: 1.9838, D(x): 0.67, D(G(z)): 0.23
Epoch [41/300], Step [400/1875], d_loss: 0.6355, g_loss: 1.6316, D(x): 0.82, D(G(z)): 0.29
Epoch [41/300], Step [600/1875], d_loss: 0.9468, g_loss: 1.9164, D(x): 0.59, D(G(z)):

...

Epoch [238/300], Step [1200/1875], d_loss: 0.9004, g_loss: 1.1524, D(x): 0.78, D(G(z)): 0.34
Epoch [238/300], Step [1400/1875], d_loss: 0.9763, g_loss: 1.4942, D(x): 0.58, D(G(z)): 0.21
Epoch [238/300], Step [1600/1875], d_loss: 1.1044, g_loss: 1.2874, D(x): 0.69, D(G(z)): 0.41
Epoch [238/300], Step [1800/1875], d_loss: 0.9709, g_loss: 1.5384, D(x): 0.66, D(G(z)): 0.31
Epoch [239/300], Step [200/1875], d_loss: 0.9297, g_loss: 1.5257, D(x): 0.76, D(G(z)): 0.36
Epoch [239/300], Step [400/1875], d_loss: 0.8407, g_loss: 0.9568, D(x): 0.68, D(G(z)): 0.27
Epoch [239/300], Step [600/1875], d_loss: 0.9821, g_loss: 1.9920, D(x): 0.66, D(G(z)): 0.23
Epoch [239/300], Step [800/1875], d_loss: 0.7460, g_loss: 1.2661, D(x): 0.73, D(G(z)): 0.24
Epoch [239/300], Step [1000/1875], d_loss: 0.9820, g_loss: 0.9869, D(x): 0.69, D(G(z)): 0.32
Epoch [239/300], Step [1200/1875], d_loss: 1.2057, g_loss: 1.4410, D(x): 0.72, D(G(z)): 0.44
Epoch [239/300], Step [1400/1875], d_loss: 0.9949, g_loss: 1.4283, D(x): 0

Epoch [248/300], Step [1000/1875], d_loss: 0.8271, g_loss: 1.3904, D(x): 0.67, D(G(z)): 0.23
Epoch [248/300], Step [1200/1875], d_loss: 1.2093, g_loss: 1.2236, D(x): 0.61, D(G(z)): 0.37
Epoch [248/300], Step [1400/1875], d_loss: 0.9863, g_loss: 1.4039, D(x): 0.73, D(G(z)): 0.37
Epoch [248/300], Step [1600/1875], d_loss: 0.7827, g_loss: 1.5334, D(x): 0.78, D(G(z)): 0.32
Epoch [248/300], Step [1800/1875], d_loss: 0.9053, g_loss: 1.0368, D(x): 0.71, D(G(z)): 0.29
Epoch [249/300], Step [200/1875], d_loss: 0.9850, g_loss: 1.6365, D(x): 0.64, D(G(z)): 0.34
Epoch [249/300], Step [400/1875], d_loss: 0.8923, g_loss: 1.9331, D(x): 0.79, D(G(z)): 0.36
Epoch [249/300], Step [600/1875], d_loss: 0.9287, g_loss: 1.4824, D(x): 0.78, D(G(z)): 0.36
Epoch [249/300], Step [800/1875], d_loss: 1.0463, g_loss: 2.1615, D(x): 0.56, D(G(z)): 0.24
Epoch [249/300], Step [1000/1875], d_loss: 1.0291, g_loss: 1.7415, D(x): 0.64, D(G(z)): 0.23
Epoch [249/300], Step [1200/1875], d_loss: 0.8324, g_loss: 1.6456, D(x): 0

Epoch [258/300], Step [800/1875], d_loss: 0.9320, g_loss: 1.4429, D(x): 0.77, D(G(z)): 0.38
Epoch [258/300], Step [1000/1875], d_loss: 0.7598, g_loss: 1.8187, D(x): 0.69, D(G(z)): 0.23
Epoch [258/300], Step [1200/1875], d_loss: 0.9722, g_loss: 1.9249, D(x): 0.68, D(G(z)): 0.31
Epoch [258/300], Step [1400/1875], d_loss: 1.1897, g_loss: 1.4641, D(x): 0.60, D(G(z)): 0.29
Epoch [258/300], Step [1600/1875], d_loss: 0.7925, g_loss: 1.7483, D(x): 0.66, D(G(z)): 0.22
Epoch [258/300], Step [1800/1875], d_loss: 0.7706, g_loss: 1.7363, D(x): 0.75, D(G(z)): 0.28
Epoch [259/300], Step [200/1875], d_loss: 0.8782, g_loss: 1.5082, D(x): 0.65, D(G(z)): 0.22
Epoch [259/300], Step [400/1875], d_loss: 0.6661, g_loss: 1.9514, D(x): 0.76, D(G(z)): 0.25
Epoch [259/300], Step [600/1875], d_loss: 0.9257, g_loss: 1.6561, D(x): 0.72, D(G(z)): 0.32
Epoch [259/300], Step [800/1875], d_loss: 0.9934, g_loss: 1.5145, D(x): 0.64, D(G(z)): 0.29
Epoch [259/300], Step [1000/1875], d_loss: 0.8754, g_loss: 1.7316, D(x): 0.

Epoch [268/300], Step [600/1875], d_loss: 1.0994, g_loss: 1.5387, D(x): 0.63, D(G(z)): 0.32
Epoch [268/300], Step [800/1875], d_loss: 0.8088, g_loss: 1.5091, D(x): 0.74, D(G(z)): 0.33
Epoch [268/300], Step [1000/1875], d_loss: 0.8418, g_loss: 2.3741, D(x): 0.68, D(G(z)): 0.26
Epoch [268/300], Step [1200/1875], d_loss: 0.8857, g_loss: 1.5973, D(x): 0.79, D(G(z)): 0.35
Epoch [268/300], Step [1400/1875], d_loss: 0.9731, g_loss: 1.3667, D(x): 0.73, D(G(z)): 0.33
Epoch [268/300], Step [1600/1875], d_loss: 0.8771, g_loss: 1.7408, D(x): 0.71, D(G(z)): 0.32
Epoch [268/300], Step [1800/1875], d_loss: 0.8489, g_loss: 1.6661, D(x): 0.75, D(G(z)): 0.34
Epoch [269/300], Step [200/1875], d_loss: 1.0383, g_loss: 0.9709, D(x): 0.75, D(G(z)): 0.39
Epoch [269/300], Step [400/1875], d_loss: 0.7009, g_loss: 1.9121, D(x): 0.73, D(G(z)): 0.21
Epoch [269/300], Step [600/1875], d_loss: 0.9510, g_loss: 1.6333, D(x): 0.60, D(G(z)): 0.22
Epoch [269/300], Step [800/1875], d_loss: 0.8683, g_loss: 1.1360, D(x): 0.7

Epoch [278/300], Step [400/1875], d_loss: 0.8821, g_loss: 1.6225, D(x): 0.74, D(G(z)): 0.32
Epoch [278/300], Step [600/1875], d_loss: 0.8863, g_loss: 1.6669, D(x): 0.68, D(G(z)): 0.20
Epoch [278/300], Step [800/1875], d_loss: 0.9021, g_loss: 1.8211, D(x): 0.66, D(G(z)): 0.22
Epoch [278/300], Step [1000/1875], d_loss: 0.8694, g_loss: 1.7730, D(x): 0.69, D(G(z)): 0.29
Epoch [278/300], Step [1200/1875], d_loss: 0.9238, g_loss: 1.8009, D(x): 0.73, D(G(z)): 0.39
Epoch [278/300], Step [1400/1875], d_loss: 0.9201, g_loss: 1.4236, D(x): 0.72, D(G(z)): 0.29
Epoch [278/300], Step [1600/1875], d_loss: 1.0813, g_loss: 0.9321, D(x): 0.73, D(G(z)): 0.42
Epoch [278/300], Step [1800/1875], d_loss: 1.1958, g_loss: 1.8123, D(x): 0.70, D(G(z)): 0.37
Epoch [279/300], Step [200/1875], d_loss: 0.9882, g_loss: 1.8514, D(x): 0.67, D(G(z)): 0.30
Epoch [279/300], Step [400/1875], d_loss: 0.9081, g_loss: 1.4833, D(x): 0.78, D(G(z)): 0.37
Epoch [279/300], Step [600/1875], d_loss: 1.0804, g_loss: 1.6697, D(x): 0.6